In [1]:
import lightgbm
import pandas as pd

In [2]:
init_record = pd.read_csv("../../Data/LWB_Monthly_data.csv")


/Users/chenqinzhang/opt/anaconda3/envs/sem2_2020/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (187,188,189,191,192,193,194,195,196,197,198,200,201,202,204,205,208) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
init_record['_key_occurreddate_month']=init_record['_key_occurreddate_month'].astype("datetime64[ns]")
init_record = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])

In [20]:
init_record.shape

(123311, 345)

## Split Dataset to three part: Training, evaluation and testing data

In [4]:
import random
clientList = list(set(init_record.index.get_level_values(0)))

In [5]:
random.shuffle(clientList)

In [6]:
length = len(clientList)
ratio = int(length*0.1)
trainIDs = clientList[ratio:]
testIDs = clientList[:ratio]
train0 = init_record.drop(testIDs, level=0)
test = init_record.drop(trainIDs, level=0)

In [7]:
print(train0.shape)
print(test.shape)

(110922, 345)
(12389, 345)


In [8]:
ratio = int(len(trainIDs)*0.25)
trainRatio = trainIDs[ratio:]
evalRatio  = trainIDs[:ratio]
train = train0.drop(evalRatio, level=0)
evl = train0.drop(trainRatio,level=0)

In [9]:
print(train.shape)
print(evl.shape)
print(test.shape)

(83313, 345)
(27609, 345)
(12389, 345)


In [10]:
import numpy as np

In [11]:

def construct_feature_label(data):
    y = []
    clientList = list(set(data.index.get_level_values(0)))
    data['response_variable'] = data.groupby(level=0)['response_variable'].shift(-1)  
    data = data.groupby(level=0, as_index=False).apply(lambda group: group.iloc[:-1])
    #print(data.loc["0000D172-EA88-432F-8235-9FAA00D29072"]['response_variable'])
    y = data['response_variable']
    x = data.drop('response_variable',axis='columns')
    return x, y

x_train, y_train = construct_feature_label(train)
x_eval, y_eval = construct_feature_label(evl)

In [12]:
print(len(y_train))
print(x_train.shape)
print(len(y_eval))
print(x_eval.shape)

74264
(74264, 344)
24593
(24593, 344)


# Start training and predictions

In [13]:
from sklearn.metrics import f1_score
from sklearn import metrics

In [14]:
train_data = lightgbm.Dataset(x_train, label=y_train)
eval_data = lightgbm.Dataset(x_eval, label=y_eval)

In [15]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}


In [16]:
model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=eval_data,
                       num_boost_round=2000)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: allrows_keyworker_min_alltime_award, allrows_keyworker_min_alltime_position_name_anon_hash, allrows_keyworker_min_alltime_position_key, allrows_keyworker_min_alltime_cost_centre_key, allrows_keyworker_min_alltime_cost_centre_name_anon_hash, allrows_keyworker_min_alltime_parentkey, allrows_keyworker_min_alltime_gender, allrows_keyworker_min_alltime_person_type_key, allrows_keyworker_min_alltime_person_type_name_anon_hash, allrows_keyworker_min_alltime_postal_suburb_anon_hash, allrows_keyworker_min_alltime_postal_state_anon_hash, allrows_keyworker_min_alltime_postal_country, allrows_keyworker_min_alltime_org_level_3_key, allrows_keyworker_min_alltime_org_level_4_key, allrows_keyworker_min_alltime_employment_type_key, allrows_keyworker_min_alltime_employment_type_name, allrows_diagnosis_primary_highest_severity_diagnosis_diagnosedconditionid, allrows_diagnosis_primary_h

In [ ]:
y_hat = np.where(model.predict(x_eval) < 0.5, 0, 1)  
f1_score(y_eval,y_hat)

In [ ]:
model_cv = lightgbm.cv(parameters,
                       train_data,
                       num_boost_round=2000,return_cvbooster=True)